In [2]:
from random import randrange
from hashlib import sha1
from gmpy2 import xmpz, to_binary, invert, powmod, is_prime


def generate_p_q(L, N): #generate p,q
    g = N  # g >= 160
    n = (L - 1) // g
    b = (L - 1) % g
    while True:
        # generate q
        while True:
            s = xmpz(randrange(1, 2 ** (g)))
            a = sha1(to_binary(s)).hexdigest()
            zz = xmpz((s + 1) % (2 ** g))
            z = sha1(to_binary(zz)).hexdigest()
            U = int(a, 16) ^ int(z, 16)
            mask = 2 ** (N - 1) + 1
            q = U | mask
            if is_prime(q, 20):
                break
        # generate p
        i = 0  # counter
        j = 2  # offset
        while i < 4096:
            V = []
            for k in range(n + 1):
                arg = xmpz((s + j + k) % (2 ** g))
                zzv = sha1(to_binary(arg)).hexdigest()
                V.append(int(zzv, 16))
            W = 0
            for qq in range(0, n):
                W += V[qq] * 2 ** (160 * qq)
            W += (V[n] % 2 ** b) * 2 ** (160 * n)
            X = W + 2 ** (L - 1)
            c = X % (2 * q)
            p = X - c + 1  # p = X - (c - 1)
            if p >= 2 ** (L - 1):
                if is_prime(p, 10):
                    return p, q
            i += 1
            j += n + 1


def generate_g(p, q): #generate g
    while True:
        h = randrange(2, p - 1)
        exp = xmpz((p - 1) // q)
        g = powmod(h, exp, p)
        if g > 1:
            break
    return g


def generate_keys(g, p, q): #generate x,y
    x = randrange(2, q)  # x < q
    y = powmod(g, x, p)
    return x, y


def generate_params(L, N):
    p, q = generate_p_q(L, N)
    g = generate_g(p, q)
    return p, q, g


def sign(M, p, q, g, x):
    if not validate_params(p, q, g):
        raise Exception("Invalid params")
    while True:
        k = randrange(2, q)  # k < q
        r = powmod(g, k, p) % q
        m = int(sha1(M).hexdigest(), 16)
        try:
            s = (invert(k, q) * (m + x * r)) % q
            return r, s
        except ZeroDivisionError: #divide 0
            pass


def verify(M, r, s, p, q, g, y):
    if not validate_params(p, q, g):
        raise Exception("Invalid params")
    if not validate_sign(r, s, q):
        return False
    try:
        w = invert(s, q)
    except ZeroDivisionError:
        return False
    m = int(sha1(M).hexdigest(), 16)
    u1 = (m * w) % q
    u2 = (r * w) % q
    v = (powmod(g, u1, p) * powmod(y, u2, p)) % p % q
    if v == r:
        return True
    return False


def validate_params(p, q, g):
    if is_prime(p) and is_prime(q):
        return True
    if powmod(g, q, p) == 1 and g > 1 and (p - 1) % q:
        return True
    return False


def validate_sign(r, s, q):
    if r < 0 and r > q:
        return False
    if s < 0 and s > q:
        return False
    return True


if __name__ == "__main__":
    N = 160 # q's bits
    L = 1024 # p's bits
    p, q, g = generate_params(L, N)
    x, y = generate_keys(g, p, q)

    text = input("input the text\n")
    M = str.encode(text, "ascii")
    r, s = sign(M, p, q, g, x)
    
    if verify(M, r, s, p, q, g, y):
        print('Valid')
    else:
        print('Invalid')
        
    print(M)
    print("p = "+hex(p))
    print("q = "+hex(q))
    print("r = "+hex(r))
    print("s = "+hex(s))
    print("x = "+hex(x))
    print("y = "+hex(y))
    print("g = "+hex(g))

input the text
meow
Valid
b'meow'
p = 0x95a3b9c7ed5058ce925094a161434c8957c007a37600678a4e10fa446691ec75cfd45f9ad5839a3f529d9d606a46eadf01bb291b1f299fdebd8e72b73071d0f3e253e193a591fc1749bf5842d581933f57a13b7c0a8eb5b8208e12cdb6c1b90a991b2398d6096bfc9959eb6d0a5eb9306b2dce51078e1b4a2536ed4b205016f5
q = 0x96ed44946d997e28ba3cd13e6162c9f9321125c3
r = 0x4f83b613440081c4878abbb6fe34d32f60ef66a6
s = 0x1ccd7f6677654f090c206f736efc723e2ed5fc16
x = 0x2112c788683dfc3528badf1675b27ec609685e90
y = 0x3f6b88fac6e819a35147d2ad0e81ab3ede45b86d643c8eda67e2663486a1b89a1f37c150c57e7fded6a3eca7013a5c763f8821513be5e345636070a6f37f95c4ad907d121fc8be709248eed4880379e6086ba0f99663680cecff46d6256abb9798db3eef4e388a884af51c391de00850d8fa0eac302efb1e9eab2a4fbb071bee
g = 0x64fb2e1cfc107fe8b43d226e7417b9e1a86dad910aeacd043f04a41e5ce0d88554d2d63aa6b0e3a13bdba6bac5744786a4540a5d33dcfd65c8ee1d2d750ab28eb88d994c338de08f08ffc426e6d0cd60b43b293dd82a6bb482b3f3ab35ed58e60fcda7ed2c6eb77fddc185c22d4c2fed7cbb8a8c471c4dd1a89d98